In [3]:
# !pip install numpy
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

from sklearn.metrics import root_mean_squared_error

In [4]:
# !pip install mlflow
import mlflow


In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df.PULocationID + "_" + df.DOLocationID
    
    return df

In [6]:
df_train = read_dataframe('./data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2024-02.parquet')

print(df_train.shape[0], df_val.shape[0])

54373 51497


In [7]:
categorical = ['PU_DO', 'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'

pipeline = make_pipeline(
    DictVectorizer(),
    LinearRegression()
)

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
pipeline.fit(train_dicts, y_train)
y_pred = pipeline.predict(val_dicts)

print(f"root mean squared error is: {root_mean_squared_error(y_val, y_pred)}")

root mean squared error is: 5.926355548541141


In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)

In [ ]:
alpha = 0.001

# mlflow ui --backend-store-uri sqlite:///mlflow.db

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

with mlflow.start_run():

    mlflow.set_tag("developer", "mohammad")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2024-02.parquet")

    pipeline = make_pipeline(
        DictVectorizer(),
        Lasso(alpha)
    )
    
    mlflow.log_param("alpha", alpha)

    pipeline.fit(train_dicts, y_train)

    y_pred = pipeline.predict(val_dicts)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="./models/lin_reg.bin", artifact_path="models_pickle")

### Adding hyper-parameter tuning

In [9]:
# !pip install xgboost
# !pip install hyperopt

from xgboost import XGBRegressor

import numpy as np
import random 

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

# the following uses some 'baysian approach' to find the best hyper-parameter
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [34]:
# dv = DictVectorizer()
# X_train = dv.fit_transform(train_dicts)
# X_val = dv.fit_transform(val_dicts)

# train = xgb.DMatrix(X_train, label=y_train)
# valid = xgb.DMatrix(X_val, label=y_val)

In [51]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "lasso")
        mlflow.log_params(params)

        pipeline = make_pipeline(
            DictVectorizer(),
            Lasso(alpha=params['alpha'])
        )
        pipeline.fit(train_dicts, y_train)
        y_pred = pipeline.predict(val_dicts)

        y_pred = pipeline.predict(val_dicts)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [54]:
search_space = {
    'alpha': hp.loguniform('alpha', -5, -1),
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=20,
    trials=Trials()
)

100%|██████████| 20/20 [02:07<00:00,  6.39s/trial, best loss: 7.406882414959407]


In [ ]:
def objective_xgb(params):

    # Set seeds for reproducibility
    seed = params.get('seed', 42)
    np.random.seed(seed)
    random.seed(seed)

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        model = XGBRegressor(
            # params,
            learning_rate=params['learning_rate'],
            max_depth=int(params['max_depth']),
            reg_alpha=params['reg_alpha'],
            reg_lambda=params['reg_lambda'],
            random_state=seed,
            verbosity=0,
            objective=params['objective']
        )

        pipeline = make_pipeline(
            DictVectorizer(),
            model
        )

        pipeline.fit(train_dicts, y_train)
        y_pred = pipeline.predict(val_dicts)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [57]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -6, -1),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}
best_result = fmin(
    fn=objective_xgb,
    space=search_space,
    algo=tpe.suggest,
    max_evals=20,
    trials=Trials()
)

 20%|██        | 4/20 [11:09<1:01:32, 230.77s/trial, best loss: 5.145984609368992]

: 

In [16]:
import mlflow.sklearn
import mlflow.xgboost

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")


params = {
    'learning_rate':    0.410215639489084,
    'max_depth':    9,
    'min_child_weight':    2.955225015279242,
    'objective':     'reg:squarederror',
    'reg_alpha':    0.07847140786574915,
    'reg_lambda':    0.15098298624710604,
    'seed':    42
}

model = XGBRegressor(
    learning_rate=params['learning_rate'],
    max_depth=int(params['max_depth']),
    reg_alpha=params['reg_alpha'],
    reg_lambda=params['reg_lambda'],
    random_state=params['seed'],
    verbosity=0,
    objective=params['objective']
)

pipeline = make_pipeline(
    DictVectorizer(),
    model
)

with mlflow.start_run():
    pipeline.fit(train_dicts, y_train)
    y_pred = pipeline.predict(val_dicts)
    rmse = root_mean_squared_error(y_val, y_pred)
    print(rmse)

    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)
    mlflow.log_metric("rmse", rmse)
    
    # Log the entire sklearn pipeline
    mlflow.sklearn.log_model(pipeline, "model")

2025/05/14 11:26:58 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.
2025/05/14 11:27:05 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.


5.112587189508359


2025/05/14 11:27:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# TO Test the model

In [22]:
import mlflow.pyfunc

# Load the pipeline model
model_uri = 'runs:/65a7096112cb41b693e557e8008cfa9c/model'
model = mlflow.pyfunc.load_model(model_uri)

# Predict directly on val_dicts (list of dicts)
preds = model.predict(val_dicts)

print(root_mean_squared_error(y_val, preds))

# print(preds)


2025/05/14 11:39:01 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.


5.112587189508359
